In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os

#### **검색 키워드**
검색 키워드는 *OO구 ㅁㅁㅁ*
- *OO구* : 25개 서울 지역구
- *ㅁㅁㅁ* : 각 대분류 카테고리의 중분류에 해당하는 키워드들


- **대분류 - 중분류**
    - **음식점**
      - 한식, 양식, 중식, 일식, 패스트푸드, 분식
    - **카페&디저트**
      - 카페, 디저트카페, 베이커리
    - **여가활동**
      - 전시관, 공방, 팝업스토어, 극장, 서점, 복합쇼핑몰
    - **걷기**
      - 공원, 시장, 거리

In [2]:
###### 검색 키워드 준비 ######
regions = ['용산구']

restaurant_keyword = ['한식', '양식', '중식', '일식', '패스트푸드', '분식']
cafe_keyword = ['카페', '디저트카페', '베이커리']
activity_keyword = ['전시관', '공방', '팝업스토어', '극장', '서점', '복합쇼핑몰']
waliking_keyword = ['공원', '시장', '거리']

restaurant_search = []
cafe_search = []
activity_search = []
waliking_search = []
for r in regions:
    for res in restaurant_keyword:
        temp = r+" "+res
        restaurant_search.append(temp)
    for cafe in cafe_keyword:
        temp = r+" "+cafe
        cafe_search.append(temp)
    for activity in activity_keyword:
        temp = r+" "+activity
        activity_search.append(temp)
    for waliking in waliking_keyword:
        temp = r+" "+waliking
        waliking_search.append(temp)

print("<음식점 검색 텍스트>")
print(restaurant_search)
print("-"*20)
print("<카페&디저트 검색 텍스트>")
print(cafe_search)
print("-"*20)
print("<여가활동 검색 텍스트>")
print(activity_search)
print("-"*20)
print("<걷기 검색 텍스트>")
print(waliking_search)

<음식점 검색 텍스트>
['용산구 한식', '용산구 양식', '용산구 중식', '용산구 일식', '용산구 패스트푸드', '용산구 분식']
--------------------
<카페&디저트 검색 텍스트>
['용산구 카페', '용산구 디저트카페', '용산구 베이커리']
--------------------
<여가활동 검색 텍스트>
['용산구 전시관', '용산구 공방', '용산구 팝업스토어', '용산구 극장', '용산구 서점', '용산구 복합쇼핑몰']
--------------------
<걷기 검색 텍스트>
['용산구 공원', '용산구 시장', '용산구 거리']


In [3]:
search_key = []
names = []
categories = []
ratings = []
addresses = []
tels = []
year_10 = []
year_20 = []
year_30 = []
year_40 = []
year_50 = []
year_60 = []
year_list = [year_10,year_20,year_30,year_40,year_50,year_60]
male = []
female = []
detail_url = []

In [4]:
#웹크롤링 작업
def crawling(keyword, names, categories, ratings, addresses, tels):
    ###1. frame 전환
    main_iframe = driver.find_element(By.CSS_SELECTOR,"iframe#searchIframe")
    driver.switch_to.frame(main_iframe)
    time.sleep(3)
    
    ###2. scroll하기
    scroll_div = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')
    # 맨 아래까지 스크롤 해서 해당 페이지의 내용이 다 표시되도록 함 - 6번 스크롤 하기
    for i in range(6):
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
    
    ###3. place 개수 세기
    place_lists = driver.find_elements(By.CSS_SELECTOR, 'span.place_bluelink')
    print(len(place_lists))
    pages = driver.find_elements(By.CSS_SELECTOR, 'div.zRM9F > a')
    print(len(pages))
    
    ###4. place상세정보 들어가서 정보 저장
    for i in range(len(place_lists)):
        if len(place_lists)==13:break
        #상세정보
        #place.click()
        # click()이 자꾸 안돼서 자바스크립트 코드로 실행하도록
        driver.execute_script("arguments[0].click();", place_lists[i])
        time.sleep(1)
        
        driver.switch_to.default_content()
        detail_iframe = driver.find_element(By.CSS_SELECTOR,"iframe#entryIframe")
        driver.switch_to.frame(detail_iframe)
        time.sleep(3)
        
        # 스크롤 내려서 나머지 화면 로딩
        driver.execute_script('window.scrollTo(0,8000);')
        time.sleep(0.5)
        
        # 검색글자, 상세정보url, 이름, 카테고리, 별점, 주소, 전화번호 수집
        search_key.append(keyword)
        detail_url.append(driver.current_url)
        names.append(driver.find_element(By.CSS_SELECTOR, "span.Fc1rA").text)
        categories.append(driver.find_element(By.CSS_SELECTOR, "span.DJJvD").text)
        try : 
            ratings.append(driver.find_element(By.CSS_SELECTOR, "div.dAsGb > span > em").text)
        except:
            ratings.append("")
        addresses.append(driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text)
        try:
            tels.append(driver.find_element(By.CSS_SELECTOR, 'span.dry01').text)
        except:
            tels.append("")

            
        #데이터랩 -연령별 가져오기
        #- 1 데이터랩 정보 더보기 클릭해야 하는 경우
        try:
            driver.find_element(By.CSS_SELECTOR,"div.place_section.I_y6k > div.lfH3O > a").click()#app-root > div > div > div > div:nth-child(7) > div > div.lazyload-wrapper > div.place_section.I_y6k > div.lfH3O > a
            print('클릭')
        except:
            pass
        #- 2 데이터랩 정보 더보기 클릭 없는 경우
        year_per = driver.find_elements(By.CSS_SELECTOR, "span.oozz2  > span:nth-child(1)")
        #- 2.1 데이터랩이 있는 경우
        if year_per:
            for y,p in zip(year_list,year_per):
                y.append(p.text)
            male.append(driver.find_element(By.CSS_SELECTOR, 'g.c3-chart-arc.c3-target.c3-target-male > text:nth-child(3)').text)
            female.append(driver.find_element(By.CSS_SELECTOR, 'g.c3-chart-arc.c3-target.c3-target-female > text:nth-child(3)').text)
        #- 2.2 데이터랩 없는경우
        else:
            for y in year_list:
                y.append('')
            male.append('')
            female.append('')

        
        driver.switch_to.default_content()
        driver.switch_to.frame(main_iframe)
        time.sleep(3)
        print(len(names),names)
    
    return len(pages)
        

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(5)

# 1. 네이버지도 이동
url = "https://map.naver.com/v5/"
driver.get(url)
driver.refresh()# 처음 들어갈 때 팝업 등 뜨면 새로고침해서 없애줌

# 2.검색어 입력
for keyword in restaurant_search:
    driver.find_element(By.CSS_SELECTOR, 'div.input_box>input.input_search').send_keys(Keys.CONTROL +'a')
    driver.find_element(By.CSS_SELECTOR, 'div.input_box>input.input_search').send_keys(Keys.DELETE)    
    driver.find_element(By.CSS_SELECTOR, 'div.input_box>input.input_search').send_keys(keyword)
    driver.find_element(By.CSS_SELECTOR, 'div.input_box>input.input_search').send_keys(Keys.ENTER)
    time.sleep(2)

    # 2-1. page마다 수집
    page = 1
    for p in range(6):
        print("---------",p+1,"---------")
        search_key = []
        names = []
        categories = []
        ratings = []
        addresses = []
        tels = []
        year_10 = []
        year_20 = []
        year_30 = []
        year_40 = []
        year_50 = []
        year_60 = []
        year_list = [year_10,year_20,year_30,year_40,year_50,year_60]
        male = []
        female = []
        detail_url = []
        page_count = crawling(keyword,names, categories, ratings, addresses, tels)
        print()
        if page_count>=5:
            driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(7)').click()
        else:
            driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child('+str(page_count)+')').click()
        driver.switch_to.default_content()
        
        #page마다 csv에 저장하기
        data = {
            'search_key':search_key,
            'name':names,
            'category':categories,
            'rating':ratings,
            'address':addresses,
            'tel':tels,
            'year_10':year_10,
            'year_20':year_20,
            'year_30':year_30,
            'year_40':year_40,
            'year_50':year_50,
            'year_60':year_60,
            'male':male,
            'female':female,
            'detail_url':detail_url,
        }

        df = pd.DataFrame(data)
        if not os.path.exists('output.csv'):
            df.to_csv('output.csv', index=False, mode='w', encoding='utf-8-sig')
        else:
            df.to_csv('output.csv', index=False, mode='a', encoding='utf-8-sig', header=False)
    

--------- 1 ---------
54
7
1 ['용산막창']
2 ['용산막창', '식껍 용산점']
3 ['용산막창', '식껍 용산점', '몽탄']
4 ['용산막창', '식껍 용산점', '몽탄', '양문']
5 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집']
클릭
6 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈']
7 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈', '삼각지 주식']
클릭
8 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈', '삼각지 주식', '열정도고깃집']
9 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈', '삼각지 주식', '열정도고깃집', '초원']
클릭
10 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈', '삼각지 주식', '열정도고깃집', '초원', '24시뼈다귀감자탕']
11 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈', '삼각지 주식', '열정도고깃집', '초원', '24시뼈다귀감자탕', '소와나']
12 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈', '삼각지 주식', '열정도고깃집', '초원', '24시뼈다귀감자탕', '소와나', '물고기주택']
13 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈', '삼각지 주식', '열정도고깃집', '초원', '24시뼈다귀감자탕', '소와나', '물고기주택', '한담곱창 용산본점']
14 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈', '삼각지 주식', '열정도고깃집', '초원', '24시뼈다귀감자탕', '소와나', '물고기주택', '한담곱창 용산본점', '마케집']
15 ['용산막창', '식껍 용산점', '몽탄', '양문', '섬집', '남영돈', '삼각지 주식', '열정도고깃집', '초원', '2

In [24]:
print(names)

['왕두꺼비 부대찌개', '삼미집 청담본점', '떡도리탕 강남본점', '도명골 청국장', '삼백호집 신논현점', '자매수산', '중앙실비', '중앙해장', '대우부대찌개', '강남진해장', '진미평양냉면', '호족반 청담', '농민백암순대 본점', '달맞이 본점']


In [23]:
print(year_10)

['', '25.00000000000001%', '29.255666666666652%', '10.860666666666665%', '3.917142857142826%', '8.942333333333346%', '3.7728571428571174%', '24.34366666666665%', '17.862666666666655%', '19.79666666666668%', '18.798000000000016%', '14.459999999999976%', '']


In [20]:
pd.set_option('display.max_columns', None) # 전체 열 보기
pd.set_option('display.max_rows', None) # 전체 행 보기

In [18]:
#driver.quit()


result

ValueError: arrays must all be same length